### Word2vec 모델을 학습시키고 사용해보는 것이 목표
* 나무위키 덤프 활용 - 정제 - 텍스트로 된 순수 문장 얻기
* 형태소 분석 - 형태소 분류
* 말뭉치(Corpus) 데이터 준비 
* 말뭉치를 활용해서 Word2vec 모델 학습
* Word2vec 모델 활용

In [6]:
import ijson

In [44]:
input_filename = '/Users/swkim/Data/namuwiki180326/namuwiki_20180326.json'
output_filename = '/Users/swkim/Data/namuwiki180326/_namuwiki_20180326_mini.txt'

In [56]:
input_file = open(input_filename, 'r', encoding='utf-8')
texts = ijson.items(input_file, 'item.text')

In [43]:
for index, text in zip(range(10), texts):
    print(strip_wiki_literal(text))







 
틀:다른 뜻2, other1=뺄셈, rd1=뺄셈, other2=문장부호, rd2=붙임표 ) 
[목차]
== 개요 ==
U+002D. 명칭은 HYPHEN-MINUS이며, 아스키 코드와 유니코드에서 역할이 굉장히 애매한 문자이다.
== 상세 ==
이 글자는 하이픈(hyphen)으로도 쓰일 수 있고, 빼기 기호로도 쓰일 수 있고, 간혹 en dash로도 쓰일 수 있다. 초창기의 타자기와 컴퓨터는 고정폭 글꼴을 썼기 때문에 딱히 하이픈과 빼기 기호, en dash의 구분이 필요하지 않았고, 그래서 ASCII에서는 그 셋을 따로 구분하지 않았다. 하지만 조판과 그래픽 디자인에서는 하이픈, en dash, 빼기 기호가 별도의 문자로 존재하며, 실제로 유니코드에도 하이픈(‐ U+2010 HYPHEN), en dash(– U+2013 EN DASH), 빼기 기호(− U+2212 MINUS SIGN) 셋이 따로 배당돼 있다. 그러므로 현재는 원칙적으로 이 기호를 써서는 안 된다. 이 기호를 쓰는 것은 -을 ㅡ(한글 자모, eu)로 바꾸는 거나 마찬가지이기 때문. 그런데도 hyphen-minus는 현재도 널리 쓰이는데, 이는 알다시피 키보드로 입력하기 간편하고, 대부분의 문자 코드에서 똑같은 위치에 존재하기 때문이다.
대부분의 프로그래밍 언어는 ASCII 문자만 쓰기 때문에, 뺄셈과 음수를 나타낼 때 − U+2212 MINUS SIGN을 사용하지 않고 hyphen-minus를 사용한다.
MS 워드에서는 foo - bar와 같이 단어 + 공백 + hyphen-minus + 공백 + 단어를 쓸 경우, 중간의 hyphen-minus를 자동으로 en dash로 바꿔 준다(즉 foo – bar). foo 이는 일본어 가타카나 표기에서 장음()을 사용하는 것을 수입한 것이며, 원칙적으로 굳이 표기하자면 장음 부호(ː)를 쓰는 것이 맞다. 장음 표기와 붙임표 참고. 그러나 이 쌍점은 사실상 전문적인 발음 표기법에서나 쓰이며 일상적으로 무언가, 특히 의성어를 일부러 길게 늘려 

In [34]:
import re

# We want only plain texts, so strip wiki grammer.
# Refer this link to know more about grammar. https://namu.wiki/w/%EB%82%98%EB%AC%B4%EC%9C%84%ED%82%A4:%EB%AC%B8%EB%B2%95%20%EB%8F%84%EC%9B%80%EB%A7%90

# Use regular expression to capture some pattern

# see http://stackoverflow.com/questions/2718196/find-all-chinese-text-in-a-string-using-python-and-regex
chinese = re.compile(u'[⺀-⺙⺛-⻳⼀-⿕々〇〡-〩〸-〺〻㐀-䶵一-鿃豈-鶴侮-頻並-龎]', re.UNICODE)
japanese = re.compile(u'[\u3000-\u303f\u3040-\u309f\u30a0-\u30ff\uff00-\uff9f\u4e00-\u9faf\u3400-\u4dbf]', re.UNICODE)

# hangul = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글과 띄어쓰기를 제외한 모든 글자
# hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+')  # 위와 동일
# result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거


def strip_wiki_literal(text):               
    text = re.sub(r"\{\{\{#\!html[^\}]*\}\}\}", '', text, flags=re.IGNORECASE|re.MULTILINE|re.DOTALL) # remove html
    text = re.sub(r"#redirect .*", '', text, flags=re.IGNORECASE) # remove redirect
    text = re.sub(r"\[\[분류:.*", '', text) # remove 분류
    text = re.sub(r"\[\[파일:.*", '', text) # remove 파일
    text = re.sub(r"\* 상위 문서 ?:.*", '', text) # remove 상위문서        
    text = re.sub(r"\[youtube\(\w+\)\]", '', text, flags=re.IGNORECASE) # remove youtube
    text = re.sub(r"\[include\(([^\]|]*)(\|[^]]*)?\]", r'\1', text, flags=re.IGNORECASE) # remove include
    text = re.sub(r"\[\[(?:[^\]|]*\|)?([^\]|]+)\]\]", r'\1', text) # remove link
    text = re.sub(r"\[\*([^\]]*)\]", '', text) # remove 각주
    text = re.sub(r"\{\{\{([^\ }|]*) ([^\}|]*)\}\}\}", r'\2', text) # remove text color/size
    text = re.sub(r"'''([^']*)'''", r'\1', text) # remove text bold
    text = re.sub(r"(~~|--)([^']*)(~~|--)", '', text) # remove strike-through
    text = re.sub(r"\|\|(.*)\|\|", '', text) # remove table
    text = re.sub(r"\n\s*\n*", '\n', text) # remove empty line                       
    text = chinese.sub('', text) # remove chinese
    text = japanese.sub('', text) # remove japanese
    return text


In [55]:
# output_filename = '/Users/swkim/Data/namuwiki180326/_namuwiki_20180326_mini.txt'

item_limit = 10000
minimum_length = 500

with open(output_filename, 'w', encoding='utf-8') as output_file:
    for index, text in zip(range(item_limit), texts):
        if (len(text) > minimum_length):
            try:
                a_line = strip_wiki_literal(text)
                output_file.write(a_line + '\n')
            except UnicodeEncodeError as e:
                print("UnicodeEncodeError ({0}) : {1}".format(e, text))

NameError: name 'e' is not defined